In [5]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()

In [143]:
class Lotteria:
    def __init__(self,
                 max_num=90,
                 max_sup=90,
                 length=6,
                 superstar=0):
        self.max_numeri = max_num
        self.max_superstar = max_sup
        self.length = length
        self.superstar = superstar

    @staticmethod
    def pallottoliere(maximo):
        while True:
            yield rnd.randint(1, maximo)

    def choice(self, length, num):
        numeri = list(range(1, num+1))

        estratti = []
        for _ in range(length):
            rnd.shuffle(numeri)
            estratto = rnd.choice(numeri)
            estratti.append(estratto)
            numeri.remove(estratto)

        return sorted(estratti)

    def sample(self, length, num):
        numeri = list(range(1, num+1))

        return sorted(rnd.sample(numeri, k=length))

    def int(self, length, num):
        estratti = set()
        while len(estratti) < length:
            estratti.add(next(self.pallottoliere(num)))

        return sorted(estratti)

    def many(self, extractor, length, num):
        numeri = list(range(1, num+1))
        samples = [extractor(length, num) for s in numeri]
        rnd.shuffle(samples)
        estratti = sorted(rnd.choice(samples))

        return estratti

    def __call__(self, backend='choice', many=True):
        if backend == 'choice':
            self.backend = 'choice'
            extractor = self.choice

        if backend == 'sample':
            self.backend = 'sample'
            extractor = self.sample

        if backend == 'int':
            self.backend = 'int'
            extractor = self.int

        if not many:
            estratti = extractor(self.length, self.max_numeri)

            if self.superstar:
                super = extractor(self.superstar, self.max_superstar)

        else:
            estratti = self.many(extractor, self.length, self.max_numeri)

            if self.superstar:
                super = self.many(extractor, self.superstar,
                                  self.max_superstar)

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print(f'Estrazione del: {now}', '\nNumeri estratti:', *estratti)
        if self.superstar:
            print('Superstar:', *sorted(super))

In [144]:
superenalotto = Lotteria(90, 90, 6, 2)

In [145]:
for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    superenalotto(backend=b, many=False)

print( '\n################# many #########################\n')

for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    superenalotto(backend=b, many=True)

### choice ###
Estrazione del: 27/05/2023 16:44 
Numeri estratti: 7 20 31 39 60 64
Superstar: 35 46
### int ###
Estrazione del: 27/05/2023 16:44 
Numeri estratti: 7 56 68 70 80 90
Superstar: 4 49
### sample ###
Estrazione del: 27/05/2023 16:44 
Numeri estratti: 2 4 19 49 57 86
Superstar: 28 89

################# many #########################

### choice ###
Estrazione del: 27/05/2023 16:44 
Numeri estratti: 34 41 45 55 67 87
Superstar: 70 76
### int ###
Estrazione del: 27/05/2023 16:44 
Numeri estratti: 5 10 48 50 63 79
Superstar: 29 35
### sample ###
Estrazione del: 27/05/2023 16:44 
Numeri estratti: 1 16 30 48 62 83
Superstar: 28 33


In [147]:
eurojack = Lotteria(50, 10, 5, 2)

In [148]:
for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    eurojack(backend=b, many=False)

print( '\n################# many #########################\n')

for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    eurojack(backend=b, many=True)

### choice ###
Estrazione del: 27/05/2023 16:44 
Numeri estratti: 2 3 19 27 46
Superstar: 6 10
### int ###
Estrazione del: 27/05/2023 16:44 
Numeri estratti: 7 10 11 28 41
Superstar: 4 6
### sample ###
Estrazione del: 27/05/2023 16:44 
Numeri estratti: 2 12 19 20 34
Superstar: 1 10

################# many #########################

### choice ###
Estrazione del: 27/05/2023 16:44 
Numeri estratti: 1 19 20 42 44
Superstar: 3 4
### int ###
Estrazione del: 27/05/2023 16:44 
Numeri estratti: 15 21 22 28 47
Superstar: 3 5
### sample ###
Estrazione del: 27/05/2023 16:44 
Numeri estratti: 6 15 28 30 45
Superstar: 1 5
